In [1]:
from ThesisFunctions import *

In [91]:
def model_2D(t, A_1, alpha_1, A_2, alpha_2):
    """
    Calculates the sum of two exponentials at given times.
    
    Args:
        t (array-like): The time points for the calculation.
        A_1 (float): Amplitude of the first exponential term.
        alpha_1 (float): Decay rate of the first exponential term.
        A_2 (float): Amplitude of the second exponential term.
        alpha_2 (float): Decay rate of the second exponential term.
    
    Returns:
        np.array: The calculated sum of two exponential functions.
    """
    exp_input1 = float(alpha_1) * np.array(t, dtype=float)
    exp_input2 = float(alpha_2) * np.array(t, dtype=float)
    return A_1 * np.exp(exp_input1) + A_2 * np.exp(exp_input2)


def sse_2D(params, data):
    """
    Computes the sum of squared errors between model predictions and actual data.
    
    Args:
        params (tuple): A tuple of parameters (A_1, alpha_1, A_2, alpha_2) for the model.
        data (pd.DataFrame): DataFrame containing 'Time' and 'Data' columns.
    
    Returns:
        float: The calculated sum of squared errors.
    """
    A_1, alpha_1, A_2, alpha_2 = params
    predictions = model_2D(data['Time'], A_1, alpha_1, A_2, alpha_2)
    return np.sum((data['Data'] - predictions) ** 2)


def data_gen_2D(num_data_points=4, noise_level=0.1, A_1=1, alpha_1=1, A_2=1, alpha_2=1):
    """
    Generates synthetic data based on the sum of two exponentials plus noise.
    
    Args:
        num_data_points (int): Number of data points to generate.
        noise_level (float): Standard deviation of Gaussian noise.
        A_1 (float): Amplitude of the first exponential term.
        alpha_1 (float): Decay rate of the first exponential term.
        A_2 (float): Amplitude of the second exponential term.
        alpha_2 (float): Decay rate of the second exponential term.
    
    Returns:
        pd.DataFrame: DataFrame with columns 'Time' and 'Data' containing the generated data.
    """
    t = np.arange(num_data_points)
    x = A_1 * np.exp(alpha_1 * t) + A_2 * np.exp(alpha_2 * t)
                                                 
    noise = noise_level * x *  np.random.normal(0, 1, size=t.shape) 
         
    x_noisy = x + noise    

    x_noisy_rational = np.array([min(sp.Rational(int(xn * 100), 100), 10000000) for xn in x_noisy])
    data = pd.DataFrame({'Time': t, 'Data': x_noisy_rational})
    return data

def groeb_2D(x_i, t_i):
    """
    Computes the Groebner basis for the system of equations derived from partial derivatives of SSE.
    
    Args:
        x_i (list): List of data points.
        t_i (list): List of time points corresponding to the data points.
    
    Returns:
        GroebnerBasis: The Groebner basis for the system, which simplifies solving the equations.
    """
    A_1, b_1, A_2, b_2 = sp.symbols('A_1 b_1 A_2 b_2')
    SSE_poly = sum([(x - (A_1 * b_1**t + A_2 * b_2**t))**2 for x, t in zip(x_i, t_i)])
    print(SSE_poly)
    partial_A_1 = sp.diff(SSE_poly, A_1)
    partial_b_1 = sp.diff(SSE_poly, b_1)
    partial_A_2 = sp.diff(SSE_poly, A_2)
    partial_b_2 = sp.diff(SSE_poly, b_2)
    
    B = sp.groebner([partial_A_1, partial_b_1, partial_A_2, partial_b_2], A_1, b_1, A_2, b_2, order='lex')
    
    return B

def evaluate_hessian_at_minimas_4x4(params, x_i, t_i):
    """
    Evaluates the Hessian matrix at the estimated minimas to ascertain their true nature,
    specifically determining if they are indeed minimas. This is achieved by constructing
    and analyzing a 4x4 Hessian matrix based on the second derivatives of the SSE polynomial,
    which is adjusted for a model involving combinations of parameters A_1, A_2, b_1, and b_2.

    Parameters:
    - params (list of tuples): Estimated parameter sets (A_1, A_2, b_1, b_2), each representing
      a unique combination of model parameters under consideration.
    - x_i (list): The observed data values.
    - t_i (list): The corresponding time values for the observed data.

    Returns:
    - list of tuples: Each tuple contains a parameter set and a boolean indicating if it represents
      a minimum, based on the positiveness of all eigenvalues of the Hessian matrix evaluated at those parameters.
    """
    # Define your symbols
    A_1, A_2, b_1, b_2 = sp.symbols('A_1 A_2 b_1 b_2')
    SSE_poly = sum([(x - A_1 * b_1 ** t - A_2 * b_2 ** t )**2 for x, t in zip(x_i, t_i)])  # Adjust according to your model

    # Compute the second derivatives to form the Hessian matrix
    partials = [[A_1, A_2, b_1, b_2], [A_1, A_2, b_1, b_2]]
    Hessian = sp.Matrix([[sp.diff(sp.diff(SSE_poly, i), j) for i in partials[0]] for j in partials[1]])
    #print(Hessian)

    Hessian_func = lambdify((A_1, A_2, b_1, b_2), Hessian, 'numpy')

    
    minima_results = []
    for param in params:
        # Substitute parameter values into Hessian
        #Hessian_at_point = Hessian.subs({A_1: param[0], A_2: param[1], b_1: param[2], b_2: param[3]})
        
        # Convert Hessian to a numerical matrix for eigenvalue computation
        #Hessian_num = np.array(Hessian_at_point).astype(np.float64)
        
        # Compute eigenvalues
        #eigenvalues = np.linalg.eigvals(Hessian_num)

        # Use the lambdified function to evaluate the Hessian matrix numerically at the parameter values
        Hessian_at_point = Hessian_func(*param)
        
        # Compute eigenvalues
        eigenvalues = np.linalg.eigvals(Hessian_at_point)
        print(eigenvalues)
        
        # Check if all eigenvalues are positive
        if all(val > 0 for val in eigenvalues):
            minima_results.append((param, True))
        else:
            minima_results.append((param, False))

    return minima_results

In [ ]:

# Define the noise levels to be tested
noise_levels = [0, 0.1, 0.5, 1, 1.5, 3, 5, 10, 20, 30, 40, 70, 100, 200, 300, 400, 500]

# Function to generate 2D data given specific parameters
# Placeholder for the actual function definition
def data_gen_2D(num_points, noise_level, A1_val, b1_val, A2_val, b2_val):
    # Placeholder for data generation logic
    # Should return a DataFrame or similar structure with 'Time' and 'Data' columns
    pass

# Iterate over each noise level
for level in noise_levels:
    print("[")  # Start a new array for this noise level
    
    # Generate and process 10 datasets for the current noise level
    for _ in range(10):
        # Generate data with the specified noise level and other parameters
        data = data_gen_2D(6, level, 1, 1, 1, 1)
        
        # Extract the 'Data' and 'Time' columns as numpy arrays
        x_i = np.array(data['Data'].values)
        t_i = np.array(data['Time'].values)
        
        # Uncomment the line below to visualize each dataset as a scatter plot
        # plt.scatter(t_i, x_i)
        
        # Define the symbolic variables for SSE calculation
        A_1, b_1, A_2, b_2 = sp.symbols('A_1 b_1 A_2 b_2')
        
        # Construct the SSE polynomial based on the generated dataset
        SSE_poly = sum([(x - (A_1 * b_1**t + A_2 * b_2**t))**2 for x, t in zip(x_i, t_i)])
        
        # Convert the polynomial to a string, replacing '**' with '^' for readability
        SSE_poly_str = str(SSE_poly).replace('**', '^')
        
        # Print the polynomial, followed by a comma and a newline for formatting
        print(SSE_poly_str + ", ")
        print(" ")
    
    print("]")  # Close the array for this noise level


In [ ]:
#Tests 

In [ ]:
solutions_9 = Solution: [1.9999412387576216, -3.683403866095807e-50, 2.718291518967122, 0.009048866408291877]
Solution: [-4.5800496072139865e-46, 1.9999412387576216, 0.009048866408291877, 2.718291518967122]
Solution: [-0.0031971412701424927, 2.0000125174957404, 1.1836730766667922, 2.7182800658950907]
Solution: [2.0000125174957404, -0.0031971412701424927, 2.7182800658950907, 1.1836730766667922] 

In [ ]:
solutions_8 = Solution: [-0.003283745663122025, 1.9999902330549197, 1.1415272798786824, 2.718284060960978]
Solution: [1.9999902330549197, -0.003283745663122025, 2.718284060960978, 1.1415272798786824]
Solution: [1.9998645646697784, -5.449934291672368e-51, 2.7183071039852056, 0.021154825565212355]
Solution: [3.286329556593869e-34, 1.9998645646697784, 0.021154825565212355, 2.7183071039852056]

In [ ]:
solutions_7 = Solution: [-0.003270868639170967, 2.00010873388842, 1.2544070909848721, 2.718260037405]
Solution: [1.9997492716379965, 2.8548642403301784e-38, 2.7183346609639636, 0.040058952926207626]
Solution: [2.00010873388842, -0.003270868639170967, 2.718260037405, 1.2544070909848721]
Solution: [-6.603706857728832e-36, 1.9997492716379965, 0.040058952926207626, 2.7183346609639636]

In [ ]:
solutions_6 = [2.00010873388842, -0.003270868639170967, 2.718260037405, 1.2544070909848721],
Solution: [1.9997492716379965, 3.3435295249646144e-38, 2.7183346609639636, 0.040058952926207626]
Solution: [-0.003270868639170967, 2.00010873388842, 1.2544070909848721, 2.718260037405]
Solution: [-2.418852062633437e-49, 1.9997492716379965, 0.040058952926207626, 2.7183346609639636]

In [53]:
solutions_5 = [
                [1.9995018213071998, -9.23983112668591e-52, 2.718406429546815, 0.08414083486861897],
               [1.2366108735154307e-36, 1.9995018213071998, 0.08414083486861897, 2.718406429546815],
               [2.000909509169935, -0.003996760161307073, 2.7180891281188666, 1.5544251754573302],
               [-0.003996760161307073, 2.000909509169935, 1.5544251754573302, 2.7180891281188666]
              ]

In [54]:
solutions_4 = [[1.9990925083061657, -3.6297006103241793e-53, 2.7185573311414557, 0.1723791385773471],
               [-3.5627284780257025e-39, 1.9990925083061657, 0.1723791385773471, 2.7185573311414557]]

In [55]:
data = data_gen_2D(4, 0, 1, 1, 1, 1)

x_i = np.array(data['Data'].values)
t_i = np.array(data['Time'].values)

In [56]:
eval = evaluate_hessian_at_minimas_4x4(solutions_4, x_i, t_i)

[ 5.09013018e+03  1.67426455e+01  1.63991810e+00 -7.79197360e-05]
[ 5.09013018e+03  1.67426455e+01  1.63991810e+00 -7.79197360e-05]


In [57]:
eval

[([1.9990925083061657,
   -3.6297006103241793e-53,
   2.7185573311414557,
   0.1723791385773471],
  False),
 ([-3.5627284780257025e-39,
   1.9990925083061657,
   0.1723791385773471,
   2.7185573311414557],
  False)]

In [110]:
noise_levels = [0, 0.1, 0.5, 1, 1.5, 3, 5, 10, 20, 30, 40, 70, 100, 200, 300, 400, 500]

for level in noise_levels:
    
    print("[") 
    for _ in range(10):
        data = data_gen_2D(6, level, 1, 1, 1, 1)
        x_i = np.array(data['Data'].values)
        t_i = np.array(data['Time'].values)
        #plt.scatter(t_i, x_i)
        A_1, b_1, A_2, b_2 = sp.symbols('A_1 b_1 A_2 b_2')
        SSE_poly = sum([(x - (A_1 * b_1**t + A_2 * b_2**t))**2 for x, t in zip(x_i, t_i)])
        SSE_poly_str = str(SSE_poly).replace('**', '^')
    
        print(SSE_poly_str + ", ")
        print(" ")
    
    print("]")

[
(-A_1 - A_2 + 2)^2 + (-A_1*b_1 - A_2*b_2 + 543/100)^2 + (-A_1*b_1^2 - A_2*b_2^2 + 1477/100)^2 + (-A_1*b_1^3 - A_2*b_2^3 + 4017/100)^2 + (-A_1*b_1^4 - A_2*b_2^4 + 10919/100)^2 + (-A_1*b_1^5 - A_2*b_2^5 + 14841/50)^2, 
 
(-A_1 - A_2 + 2)^2 + (-A_1*b_1 - A_2*b_2 + 543/100)^2 + (-A_1*b_1^2 - A_2*b_2^2 + 1477/100)^2 + (-A_1*b_1^3 - A_2*b_2^3 + 4017/100)^2 + (-A_1*b_1^4 - A_2*b_2^4 + 10919/100)^2 + (-A_1*b_1^5 - A_2*b_2^5 + 14841/50)^2, 
 
(-A_1 - A_2 + 2)^2 + (-A_1*b_1 - A_2*b_2 + 543/100)^2 + (-A_1*b_1^2 - A_2*b_2^2 + 1477/100)^2 + (-A_1*b_1^3 - A_2*b_2^3 + 4017/100)^2 + (-A_1*b_1^4 - A_2*b_2^4 + 10919/100)^2 + (-A_1*b_1^5 - A_2*b_2^5 + 14841/50)^2, 
 
(-A_1 - A_2 + 2)^2 + (-A_1*b_1 - A_2*b_2 + 543/100)^2 + (-A_1*b_1^2 - A_2*b_2^2 + 1477/100)^2 + (-A_1*b_1^3 - A_2*b_2^3 + 4017/100)^2 + (-A_1*b_1^4 - A_2*b_2^4 + 10919/100)^2 + (-A_1*b_1^5 - A_2*b_2^5 + 14841/50)^2, 
 
(-A_1 - A_2 + 2)^2 + (-A_1*b_1 - A_2*b_2 + 543/100)^2 + (-A_1*b_1^2 - A_2*b_2^2 + 1477/100)^2 + (-A_1*b_1^3 - A_2*b_2^3 + 